>TFLearn
<table>
    <tr>
        <td>Function</td> <td>Description</td>
    </tr>
      <tr>
<td>evaluate(X, Y, batch_size=128)</td> <td>Perform evaluations of the model on given samples.</td>
       </tr>
          <tr>
<td>fit(X, Y, n_epoch=10)</td> <td>Train the model with input features X and target Y to the network.</td>
           </tr>
   <tr>
<td>get_weights(weight_tensor)</td> <td>Get a variable’s weights.</td>
       </tr>
       <tr>
<td>load(model_file)</td> <td>Restore model weights.</td>
           </tr>
       <tr>
<td>predict(X)</td> <td>Get model predictions for the given input data.</td>
       </tr>
       <tr>
<td>save(model_file)</td> <td>Save model weights.</td>
       </tr>
       <tr>
<td>set_weights(tensor,weights)</td> <td>Assign a tensor variable a given value.</td>
       </tr>
</table>

In [1]:
#CNN
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression
import numpy as np
# Data loading and basic transformations
import tflearn.datasets.mnist as mnist
X, Y, X_test, Y_test = mnist.load_data(one_hot=True)
X = X.reshape([-1, 28, 28, 1])
X_test = X_test.reshape([-1, 28, 28, 1])

# Building the network
CNN = input_data(shape=[None, 28, 28, 1], name='input')
CNN = conv_2d(CNN, 32, 5, activation='relu', regularizer="L2")
CNN = max_pool_2d(CNN, 2)
CNN = local_response_normalization(CNN)
CNN = conv_2d(CNN, 64, 5, activation='relu', regularizer="L2")
CNN = max_pool_2d(CNN, 2)
CNN = local_response_normalization(CNN)
CNN = fully_connected(CNN, 1024, activation=None)
CNN = dropout(CNN, 0.5)
CNN = fully_connected(CNN, 10, activation='softmax')
CNN = regression(CNN, optimizer='adam', learning_rate=0.0001,loss='categorical_crossentropy', name='target')

# Training the network
model = tflearn.DNN(CNN,tensorboard_verbose=0,
                    tensorboard_dir = 'MNIST_tflearn_board',checkpoint_path = 'MNIST_tflearn_checkpoints\checkpoint')
model.fit({'input': X}, {'target': Y}, n_epoch=3, 
          validation_set=({'input': X_test}, {'target': Y_test}),snapshot_step=1000,show_metric=True, run_id='convnet_mnist')

evaluation = model.evaluate({'input': X_test},{'target': Y_test})
print(evaluation)
pred = model.predict({'input': X_test})
print((np.argmax(Y_test,1)==np.argmax(pred,1)).mean())
                    
                    
'''
In TFLearn, each iteration is a full pass (forward and backward) over one
example. The training step is the number of full passes to perform, determined
by the batch size you set (the default is 64), and an epoch is a full pass over all
the training examples (50,000 in the case of MNIST).
'''

Training Step: 2579  | total loss: 0.13323 | time: 17.469s
| Adam | epoch: 003 | loss: 0.13323 - acc: 0.9706 -- iter: 54976/55000
Training Step: 2580  | total loss: 0.12498 | time: 18.652s
| Adam | epoch: 003 | loss: 0.12498 - acc: 0.9720 | val_loss: 0.04346 - val_acc: 0.9868 -- iter: 55000/55000
--
INFO:tensorflow:C:\Users\xcm\Python\QQ_PYTHON_NLP\MNIST_tflearn_checkpoints\checkpoint-2580 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:C:\Users\xcm\Python\QQ_PYTHON_NLP\MNIST_tflearn_checkpoints\checkpoint-2580 is not in all_model_checkpoint_paths. Manually adding it.
[0.9868]
0.9868


'\nIn TFLearn, each iteration is a full pass (forward and backward) over one\nexample. The training step is the number of full passes to perform, determined\nby the batch size you set (the default is 64), and an epoch is a full pass over all\nthe training examples (50,000 in the case of MNIST).\n'

In [1]:
#RNN
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences
from tflearn.datasets import imdb
# IMDb dataset loading
train, test, _ = imdb.load_data(path='imdb.pkl', n_words=10000,valid_portion=0.1)
X_train, Y_train = train
X_test, Y_test = test
'''
by equalizing the sequences with zero-padding by using
tflearn.data_utils.pad_sequences() and setting 100 as the maximum sequence length
'''
X_train = pad_sequences(X_train, maxlen=100, value=0.)
X_test = pad_sequences(X_test, maxlen=100, value=0.)
Y_train = to_categorical(Y_train, nb_classes=2)
Y_test = to_categorical(Y_test, nb_classes=2)

RNN = tflearn.input_data([None, 100]) #100的长度
RNN = tflearn.embedding(RNN, input_dim=10000, output_dim=128)

#LSTM
RNN = tflearn.lstm(RNN, 128, dropout=0.8)
RNN = tflearn.fully_connected(RNN, 2, activation='softmax')
RNN = tflearn.regression(RNN, optimizer='adam', learning_rate=0.001, loss='categorical_crossentropy')
# Training the network
model = tflearn.DNN(RNN, tensorboard_verbose=0)
model.fit(X_train, Y_train, validation_set=(X_test, Y_test), show_metric=True, batch_size=32) #n_epoch=10 默认

Training Step: 7039  | total loss: 0.02698 | time: 39.781s
| Adam | epoch: 010 | loss: 0.02698 - acc: 0.9926 -- iter: 22496/22500
Training Step: 7040  | total loss: 0.02610 | time: 40.984s
| Adam | epoch: 010 | loss: 0.02610 - acc: 0.9934 | val_loss: 0.92076 - val_acc: 0.8084 -- iter: 22500/22500
--
